### Google Scanned Objects Converter
- This Google's `Scanned Objects Dataset` contains 1030 household objects. 
    -  https://research.google/blog/scanned-objects-by-google-research-a-dataset-of-3d-scanned-common-household-items/
- However, due to having the same naming convention for each object, it is difficult to make an xml file with multiple objects.
- This notebook (converter.ipynb) converts xml and obj files to have different names so that they can easily be added to a single mjcf file.
- We will be assuming that we have downloaded xml and obj files from the following link:
    - https://github.com/kevinzakka/mujoco_scanned_objects
- Put this notebook inside the root folder (i.e., `/mujoco_scanned_objects/`)

In [1]:
import os
import shutil
import glob
import xml.etree.ElementTree as ET
print ("Ready.")

Ready.


In [2]:
# base asset paths
original_model_path  = './models'
converted_model_path = './converted_models'
converted_mesh_path  = './converted_meshes'
print ("Ready.")

Ready.


#### Remove existing working folders

In [3]:
if os.path.exists(converted_model_path):
    shutil.rmtree(converted_model_path)
    print("[%s] removed."%(converted_model_path))
if os.path.exists(converted_mesh_path):
    shutil.rmtree(converted_mesh_path)
    print("[%s] removed."%(converted_mesh_path))
print ("Done.")

[./converted_models] removed.
[./converted_meshes] removed.
Done.


#### Create working folders

In [4]:
# create processed folder if it does not exist
os.makedirs(converted_model_path, exist_ok=True)
print ("[%s] created."%(converted_model_path))
# create mesh folder in processed path
os.makedirs(converted_mesh_path, exist_ok=True)
print ("[%s] created."%(converted_mesh_path))
print ("Ready.")

[./converted_models] created.
[./converted_meshes] created.
Ready.


#### Main Loop

In [5]:
# iterate through each folder in the original_model_path
verbose_detail = False
for folder_name in os.listdir(original_model_path):
    folder_path = os.path.join(original_model_path, folder_name)
    
    if os.path.isdir(folder_path): 
        print ("\nfolder_name: [%s]"%(folder_name))
        
        # create corresponding folder in converted_model_path
        processed_folder_path = os.path.join(converted_model_path, folder_name)
        os.makedirs(processed_folder_path, exist_ok=True)
        
        # Prefix for files if folder_name starts with a digit
        prefix = 'model_' if folder_name[0].isdigit() else ''
        """
            Step 1: 
            # Copy and rename 'model.xml' to 'converted_model_path/folder_name.xml'
            Copy and rename 'model.xml' to 'converted_model_path/model.xml'
        """
        xml_file = os.path.join(folder_path, 'model.xml')
        # new_xml_file = os.path.join(processed_folder_path, f'{prefix}{folder_name}.xml')
        new_xml_file = os.path.join(processed_folder_path, f'model.xml')
        if os.path.exists(xml_file):
            shutil.copy(xml_file, new_xml_file)
            if verbose_detail: print(" Copy [%s] to [%s]"%(xml_file,new_xml_file))

        """
            Step 2: 
            Copy and rename 'model_collision_XX.obj' to 'converted_mesh_path/folder_name_collision_XX.obj'
        """
        obj_files = glob.glob(os.path.join(folder_path, 'model_collision_*.obj'))
        for obj_idx,obj_file in enumerate(obj_files):
            obj_file_name = os.path.basename(obj_file)
            new_obj_file_name = obj_file_name.replace('model', f'{prefix}{folder_name}')
            new_obj_file = os.path.join(converted_mesh_path, new_obj_file_name)
            shutil.copy(obj_file, new_obj_file)
            if verbose_detail: print("  [%d] Copy [%s] to [%s]"%(obj_idx,obj_file,new_obj_file))

        """
            Step 3: 
            Copy and rename 'texture.png' to 'converted_mesh_path/folder_name.png'
        """
        texture_file = os.path.join(folder_path, 'texture.png')
        new_texture_file = os.path.join(converted_mesh_path, f'{prefix}{folder_name}.png')
        if os.path.exists(texture_file):
            shutil.copy(texture_file, new_texture_file)
            if verbose_detail: print(" Copy [%s] to [%s]"%(texture_file,new_texture_file))

        """
            Step 4: 
            Copy and rename 'model.obj' to 'converted_model_path/folder_name.obj'
        """
        model_obj_file = os.path.join(folder_path, 'model.obj')
        new_model_obj_file = os.path.join(processed_folder_path, f'{prefix}{folder_name}.obj')
        if os.path.exists(model_obj_file):
            shutil.copy(model_obj_file, new_model_obj_file)
            if verbose_detail: print(" Copy [%s] to [%s]"%(model_obj_file,new_model_obj_file))

        """
           Step 5:
           Copy 'converted_model_path/folder_name.obj' to 'converted_mesh_path/folder_name.obj'
        """
        if os.path.exists(new_model_obj_file):
            shutil.copy(new_model_obj_file, converted_mesh_path)
            if verbose_detail: print(" Copy [%s] to [%s]"%(new_model_obj_file,converted_mesh_path))

        """ 
            Step 6: 
            Modify the contents of 'converted_model_path/folder_name.xml'
             - "model" -> "folder_name"
             - "texture" -> "folder_name"
             - "material_" -> "folder_name_"
             - Add <freejoint/>
             - Modify body name
        """
        if os.path.exists(new_xml_file):
            tree = ET.parse(new_xml_file)
            root = tree.getroot()
            
            for elem in root.iter():
                for key, value in elem.attrib.items():
                    if 'model' in value:
                        new_value = value.replace('model', f'{prefix}{folder_name}')
                        elem.set(key, new_value)
                    if 'texture' in value:
                        new_value = value.replace('texture', f'{prefix}{folder_name}')
                        elem.set(key, new_value)
                    if 'material_' in value:
                        new_value = value.replace('material_', f'material_{folder_name}_')
                        elem.set(key, new_value)
                
                if elem.text:
                    if 'model' in elem.text:
                        new_text = elem.text.replace('model', f'{prefix}{folder_name}')
                        elem.text = new_text
                    if 'texture' in elem.text:
                        new_text = elem.text.replace('texture', f'{prefix}{folder_name}')
                        elem.text = new_text
                    if 'material_' in elem.text:
                        new_text = elem.text.replace('material_', f'material_{folder_name}_')
                        elem.text = new_text
            
            # Add <freejoint /> after the first <body> inside <worldbody>
            for worldbody in root.iter('worldbody'):
                first_body = worldbody.find('body')
                if first_body is not None:
                    freejoint = ET.Element('freejoint')
                    first_body.insert(1, freejoint)

            # Modify body_name to 'body_obj_'+folder_name
            body = worldbody.find('body')
            if body is not None:
                new_body_name = 'body_obj_'+folder_name
                body.set('name',new_body_name)
            else:
                print ("body NOT FOUND")

            # Modify texxure and mesh file path
            """
            <texture file="XXX.png"/> => <texture file="prefix/XXX.png"/>
            <mesh file="XXX.png"/> => <texture file="prefix/XXX.png"/>
            """
            prefix = '../../%s/'%(converted_mesh_path)
            for mesh in root.findall(".//texture"):
                file_path = mesh.get('file')
                if file_path:
                    mesh.set('file', prefix + file_path)
            for mesh in root.findall(".//mesh"):
                file_path = mesh.get('file')
                if file_path:
                    mesh.set('file', prefix + file_path)
            

            # Save
            tree.write(new_xml_file)
            print(" Saved [%s]"%(new_xml_file))

print ("Done.")

FileNotFoundError: [Errno 2] No such file or directory: './models'